<a href="https://colab.research.google.com/github/soyRex-codes/New_LLM_Bommer_Bot-450/blob/main/PFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- 1. Reset and Install All Required Libraries ---
# For a guaranteed clean start, go to the menu and select:
# 'Runtime' -> 'Factory reset runtime'
# Then, run this cell.

# Uninstall conflicting packages from the default Colab environment
!pip uninstall -y opencv-python opencv-python-headless thinc opencv-contrib-python

# Pin NumPy to a compatible version for ChromaDB
!pip install "numpy<2.0.0"

# Install the latest core libraries as recommended for Gemma 2
!pip install -U "transformers>=4.42.3" "accelerate>=0.31.0" "torch"

# Install the other necessary libraries
!pip install -qU "bitsandbytes>=0.42.0" "sentence-transformers>=2.2.2" "chromadb>=0.4.24"

print("\n✅ All libraries have been installed in a compatible order.")
print("CRITICAL: A runtime restart is required for these changes to take effect.")
print("Please go to the menu -> 'Runtime' -> 'Restart session'.")

Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-python-headless 4.12.0.88
Uninstalling opencv-python-headless-4.12.0.88:
  Successfully uninstalled opencv-python-headless-4.12.0.88
Found existing installation: thinc 8.3.6
Uninstalling thinc-8.3.6:
  Successfully uninstalled thinc-8.3.6
Found existing installation: opencv-contrib-python 4.12.0.88
Uninstalling opencv-contrib-python-4.12.0.88:
  Successfully uninstalled opencv-contrib-python-4.12.0.88
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 85.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 151.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.

In [1]:
# --- 2. Build the Knowledge Base (No Changes Needed) ---
import json
import chromadb
from sentence_transformers import SentenceTransformer
import uuid
from google.colab import files

def load_and_build_from_json_qa(collection_name="knowledge_base_collection"):
    print("Please upload your `knowledge_base.json` file.")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded. Aborting.")
        return None
    file_name = next(iter(uploaded))
    if not file_name.lower().endswith('.json'):
        print("❌ Error: The uploaded file is not a .json file.")
        return None
    print(f"Processing '{file_name}' and building the vector database...")
    try:
        with open(file_name, 'r') as f:
            qa_data = json.load(f)
    except Exception as e:
        print(f"❌ Error reading or parsing JSON file: {e}")
        return None
    documents_to_embed = [
        f"Question: {item.get('question_variants', [''])[0]}\nAnswer: {item.get('context_answer', '')}"
        for item in qa_data if item.get("question_variants", [""])[0] and item.get("context_answer", "")
    ]
    ids_for_db = [item.get("id", str(uuid.uuid4())) for item in qa_data if item.get("question_variants", [""])[0] and item.get("context_answer", "")]
    if not documents_to_embed:
        print("⚠️ No documents were created from the JSON file. Please check its structure.")
        return None
    print(f"Created {len(documents_to_embed)} text blocks to be embedded.")
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
    doc_embeddings = embedding_model.encode(documents_to_embed, show_progress_bar=True)
    client = chromadb.Client()
    if collection_name in [c.name for c in client.list_collections()]:
        client.delete_collection(name=collection_name)
    collection = client.create_collection(name=collection_name)
    collection.add(embeddings=doc_embeddings.tolist(), documents=documents_to_embed, ids=ids_for_db)
    print(f"✅ Successfully added {collection.count()} documents to the '{collection_name}' collection.")
    return collection

collection = load_and_build_from_json_qa()

Please upload your `knowledge_base.json` file.


Saving missouri_state_university.json to missouri_state_university.json
Processing 'missouri_state_university.json' and building the vector database...
Created 4 text blocks to be embedded.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Successfully added 4 documents to the 'knowledge_base_collection' collection.


In [ ]:
# --- 3. The Complete RAG Agent with Gemma-2 ---
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
from huggingface_hub import notebook_login

gc.collect()
torch.cuda.empty_cache()
print("Logging in to Hugging Face Hub...")
notebook_login()

print("Loading the main language model (Gemma-2 9B)...")
# MODEL CHANGE: Using the Gemma-2 9B model
model_id = "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
rag_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)
print("Loading the embedding model for retrieval...")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_model_retriever = SentenceTransformer('all-MiniLM-L6-v2', device=device)
print("✅ Models loaded successfully.")

def retrieve_context(query, db_collection, embedder, n_results=3):
    query_embedding = embedder.encode(query).tolist()
    results = db_collection.query(query_embeddings=[query_embedding], n_results=n_results)
    return "\n---\n".join(results['documents'][0])

def generate_rag_response(query, db_collection):
    context = retrieve_context(query, db_collection, embedding_model_retriever)
    # MODIFICATION: Removing the system role message
    messages = [
    {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"},
    ]

    outputs = rag_pipeline(messages, max_new_tokens=512)

    # PARSING CHANGE: Using the correct output format for Gemma-2 pipeline
    # The pipeline returns the full conversation history. The assistant's final message is the last one.
    assistant_response = outputs[0]["generated_text"][-1]["content"].strip()

    return assistant_response

if 'collection' not in locals() or collection is None:
    print("\n❌ ERROR: The knowledge base ('collection') is not ready. Please run Step 2 successfully.")
else:
    print("\n--- ✅ Document RAG Chat Interface ---")
    print("Enter your question, or type 'quit' to exit.")
    print("="*40)
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break
        if not user_input.strip():
            continue
        try:
            print("Assistant is thinking...")
            response = generate_rag_response(user_input, collection)
            print(f"Assistant: {response}\n")
        except Exception as e:
            print(f"An unexpected error occurred: {e}\n")
    print("Session ended.")

Logging in to Hugging Face Hub...


Loading the main language model (Gemma-2 9B)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


Loading the embedding model for retrieval...
✅ Models loaded successfully.

--- ✅ Document RAG Chat Interface ---
Enter your question, or type 'quit' to exit.
You: hi
Assistant is thinking...
Assistant: Hello! 👋  

What can I help you with today? 😊

You: when was msu found
Assistant is thinking...
Assistant: MSU was founded on March 17, 1905.

You: how do i connect to msu wifi
Assistant is thinking...
Assistant: To connect an Android device to the MSU Wi-Fi network, go to your Wi-Fi settings and select the MSU network. When prompted for credentials, use the following settings: 

* **EAP Method:** PEAP
* **Phase 2 Authentication:** MSCHAPv2
* **CA Certificate:** 'Do Not Validate' 

Your identity should be your Missouri State Account ID (e.g., abc123), and your password is your standard university password. Leave the Anonymous Identity field blank.

You: who is boomer
Assistant is thinking...
Assistant: Boomer the Bear is the official mascot of Missouri State University! 🐻 

He's a high-